# Download Financial Statements from Companies

# 1. API connection and data download

Using the API provided by the https://sec-api.io portal to retrieve Financial Statements of companies (referred to as '10-K Annual Reports' in the United States) and convert the obtained information from XBRL format to JSON, thereby obtaining their Balance Sheet and Income Statement in a more suitable format for processing.

Install the SEC API python package.

In [1]:
!pip install sec-api --no-warn-script-location

Retrieve the API key at https://sec-api.io

In [1]:
api_key = "YOUR_API_KEY"

Install the Pandas python package.

In [3]:
!pip install pandas

Search for the company on the URL https://www.sec.gov/edgar/searchedgar/companysearch. Construct a request to the API: Select the company's 10-K report and copy its URL. Use the API endpoint for converting XBRL to JSON. Make the API request. Load the JSON into memory.

In [208]:
import requests
import json
import pandas as pd

filing_url = "https://www.sec.gov/ix?doc=/Archives/edgar/data/0001744489/000174448922000213/dis-20221001.htm"

xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"

final_url = xbrl_converter_api_endpoint + "?htm-url=" + filing_url + "&token=" + api_key

response = requests.get(final_url)

xbrl_json = json.loads(response.text)

# 2. Financial Statements into dataframes

## 2.1 Extract the Income Statement

Create a function to extract the Income Statement from the JSON file: Iterate through each element of the US GAAP that makes up the Income Statement. Consider only elements without a segment (not necessary for the analysis). Create a dataframe with the extracted information. Pivot the columns and rows of the dataframe so that US GAAP elements become the rows, and each column header represents a date range.

In [209]:
def get_income_statement(xbrl_json):
    income_statement_store = {}
    
    for usGaapItem in xbrl_json['StatementsOfIncome']:
        values = []
        indicies = []

        for fact in xbrl_json['StatementsOfIncome'][usGaapItem]:
            # Considerar solo elementos sin segmento (no son necesarios para nuestro análisis)
            if 'segment' not in fact:
                index = fact['period']['startDate'] + '-' + fact['period']['endDate']
                # Asegurarse de que no se creen duplicados de índices
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)                    

        income_statement_store[usGaapItem] = pd.Series(values, index=indicies) 

    income_statement = pd.DataFrame(income_statement_store)

    return income_statement.T

Call the function and pass the JSON file to ultimately obtain the Income Statement as a Pandas dataframe.

In [210]:
income_statement = get_income_statement(xbrl_json)
income_statement

,2019-09-29-2020-10-03,2020-10-04-2021-10-02,2021-10-03-2022-10-01
Revenues,65388000000,67418000000,82722000000
CostOfGoodsAndServicesSold,NaN,NaN,NaN
SellingGeneralAndAdministrativeExpense,12369000000,13517000000,16388000000
DepreciationDepletionAndAmortization,5345000000,5111000000,5163000000
CostsAndExpenses,61594000000,63759000000,75952000000
RestructuringSettlementAndImpairmentProvisions,5735000000,654000000,237000000
NonoperatingIncomeExpense,1038000000,201000000,-667000000
InterestIncomeExpenseNonoperatingNet,-1491000000,-1406000000,-1397000000
IncomeLossFromEquityMethodInvestments,651000000,761000000,816000000
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,-1743000000,2561000000,5285000000


Identify the index number for each column in the dataframe.

In [211]:
income_statement.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, Revenues to WeightedAverageNumberOfSharesOutstandingBasic
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   2019-09-29-2020-10-03  23 non-null     object
 1   2020-10-04-2021-10-02  23 non-null     object
 2   2021-10-03-2022-10-01  23 non-null     object
dtypes: object(3)
memory usage: 768.0+ bytes


Format the dataframe to extract the columns for the years 2021 and 2022, in that order. Display the resulting dataframe for the Income Statement.

In [212]:
income_statement_formatted = income_statement.iloc[:, [1, 0]]

income_statement_formatted

,2020-10-04-2021-10-02,2019-09-29-2020-10-03
Revenues,67418000000,65388000000
CostOfGoodsAndServicesSold,NaN,NaN
SellingGeneralAndAdministrativeExpense,13517000000,12369000000
DepreciationDepletionAndAmortization,5111000000,5345000000
CostsAndExpenses,63759000000,61594000000
RestructuringSettlementAndImpairmentProvisions,654000000,5735000000
NonoperatingIncomeExpense,201000000,1038000000
InterestIncomeExpenseNonoperatingNet,-1406000000,-1491000000
IncomeLossFromEquityMethodInvestments,761000000,651000000
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest,2561000000,-1743000000


## 2.2 Extract the Balance Sheet

Create a function to extract the Balance Sheet from the JSON file: Iterate through each element of the US GAAP that makes up the Balance Sheet. Consider only elements without a segment (not necessary for the analysis), avoid duplicate indices and add 0 if the value is null. Create a dataframe with the extracted information. Pivot the columns and rows of the dataframe so that US GAAP elements become the rows, and each column header represents a point in time.

In [213]:
def get_balance_sheet(xbrl_json):
    balance_sheet_store = {}

    for usGaapItem in xbrl_json['BalanceSheets']:
        values = []
        indicies = []

        for fact in xbrl_json['BalanceSheets'][usGaapItem]:

            if 'segment' not in fact:
                index = fact['period']['instant']

                if index in indicies:
                    continue

                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])

                indicies.append(index)                    

            balance_sheet_store[usGaapItem] = pd.Series(values, index=indicies) 

    balance_sheet = pd.DataFrame(balance_sheet_store)

    return balance_sheet.T

Call the function and pass the JSON file to ultimately obtain the Balance Sheet as a Pandas dataframe.

In [214]:
balance_sheet = get_balance_sheet(xbrl_json)
balance_sheet

,2019-09-28,2020-10-03,2021-10-02,2022-10-01
CashAndCashEquivalentsAtCarryingValue,NaN,17914000000,15959000000,11615000000
ReceivablesNetCurrent,NaN,NaN,13367000000,12652000000
InventoryNet,NaN,NaN,1331000000,1742000000
LicensedContentCostsAndAdvances,NaN,NaN,2183000000,1890000000
OtherAssetsCurrent,NaN,NaN,817000000,1199000000
AssetsCurrent,NaN,NaN,33657000000,29098000000
ProducedAndLicensedContentCosts,NaN,NaN,29549000000,35777000000
LongTermInvestments,NaN,NaN,3935000000,3218000000
PropertyPlantAndEquipmentGross,NaN,NaN,64892000000,66998000000
AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment,NaN,NaN,37920000000,39356000000


Identify the index number for each column in the dataframe.

In [215]:
balance_sheet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, CashAndCashEquivalentsAtCarryingValue to LiabilitiesAndStockholdersEquity
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   2019-09-28  1 non-null      object
 1   2020-10-03  3 non-null      object
 2   2021-10-02  36 non-null     object
 3   2022-10-01  36 non-null     object
dtypes: object(4)
memory usage: 1.4+ KB


Format the dataframe to extract the columns for the years 2021 and 2022, in that order. Display the resulting dataframe for the Balance Sheet.

In [216]:
balance_sheet_formatted = balance_sheet.iloc[:, [2, 3]]

balance_sheet_formatted

,2021-10-02,2022-10-01
CashAndCashEquivalentsAtCarryingValue,15959000000,11615000000
ReceivablesNetCurrent,13367000000,12652000000
InventoryNet,1331000000,1742000000
LicensedContentCostsAndAdvances,2183000000,1890000000
OtherAssetsCurrent,817000000,1199000000
AssetsCurrent,33657000000,29098000000
ProducedAndLicensedContentCosts,29549000000,35777000000
LongTermInvestments,3935000000,3218000000
PropertyPlantAndEquipmentGross,64892000000,66998000000
AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment,37920000000,39356000000


# 3. Save the dataframes to CSV files

Specify the path and name of the CSV files. Save the Income Statement and Balance Sheet dataframes to separated CSV files.

In [217]:
path = './Financial_Statements/'
company_name = 'WaltDisney'
statement_name_1 = 'income_statement_'
statement_name_2 = 'balance_sheet_'
type_file = '.csv'

income_statement_formatted.to_csv(path+statement_name_1+company_name+type_file)

balance_sheet_formatted.to_csv(path+statement_name_2+company_name+type_file)